In [ ]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [ ]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5 # 강의에서는 1000으로 진행하는데, 실습을 위해 에포크 낮춤.
no_cuda = False
log_interval = 100

# **Modle**

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# **Preprocess**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
train_paths = glob('/content/drive/MyDrive/dataset/mnist_png/training/*/*.png')[:1000]
test_paths = glob('/content/drive/MyDrive/dataset/mnist_png/testing/*/*.png')[:1000]
len(train_paths), len(test_paths)

참고: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class

In [ ]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        image = Image.open(path).convert("L")
        label = int(path.split('/')[-2]) # 맥의 경우 경로가 '/'로 구분
        #label = int(path.split('\\')[-2]) # 윈도우의 경우 경로가 '\'로 구분되므로 '\\'로 표기
        
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False
)

In [ ]:
for i, data in enumerate(train_loader):
  if i == 0:
    print(data[0].shape, data[1].shape)
    break

# **Optimization**

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# **Training**

In [ ]:
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter()

In [ ]:
for epoch in range(1, epochs + 1):
  # Train Mode
  model.train()

  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)

    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target) #https://pytorch.org/docs/stab/nn.html#nll-loss

    if batch_idx % log_interval == 0:
      print('Train Epoch: {}[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()))
      
      # Test mode
      model.eval()
      test_loss = 0
      correct = 0
      with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

      test_loss /= len(test_loader.dataset)
    
      accuracy = 100. * correct / len(test_loader.dataset)
    
      print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
          test_loss, correct, len(test_loader.dataset),
          accuracy))

# **Save Model**
- 공식 홈페이지에서는 바로 다음 단계에서 배울 전체 모델을 학습하는 것보다는 이 방법을 추천
- 나중에 다른 모델에 적용하거나 수정하는 등 여러가지 이유로 용이하기 때문이다.
https://pytorch.org/tutorials/begineer/saving_loading_models.html

In [ ]:
save_path = '/content/drive/MyDrive/model_weight.pt'

In [ ]:
torch.save(model.state_dict(), save_path)

In [ ]:
model.state_dict()['conv1.weight'].shape

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
model = Net().to(device)

In [ ]:
weight_dict = torch.load('/content/drive/MyDrive/model_weight.pt')

In [ ]:
weight_dict['conv1.weight'].shape

In [ ]:
model.load_state_dict(weight_dict)

In [ ]:
model.eval()

# Save Entire model

In [ ]:
save_path = '/content/drive/MyDrive/model.pt'

In [ ]:
torch.save(model, save_path)

# Load Model

In [ ]:
model = torch.load(save_path)

In [ ]:
model.eval

# Save, Load and Resuming Training

In [ ]:
checkpoint_path = '/content/drive/MyDrive/model.pt'

In [ ]:
torch.save ({
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss

}, checkpoint_path)

In [ ]:
model = Net().to(device)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [ ]:
checkpoint = torch.load(checkpoint_path)

In [ ]:
checkpoint.keys()

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [ ]:
model

In [ ]:
optimizer

In [ ]:
epoch

In [ ]:
loss

In [ ]:
model.train()